# Experiment Lifting

### Load Model & Dataset

In [1]:
import os
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from src.models.simple_3d_model import LinearModel
import src.bug_dataset
from src.real_bug_dataset import RealBugDataset
from src.eval.accuracies import keypoint_depth_pck

import torch
from torchvision import transforms
from torch.utils.data import DataLoader

In [2]:
# Plotting Code
%matplotlib widget
import matplotlib.pyplot as plt

def plot_stick_bug(ax, points, vis, prediction=False):
    limb_ranges=[[0,7],[8,14],[15,21],[22,28],[29,35],[36,42],[43,49],[53,56],[59,62]]
    reduced_limb_ranges = [[0,4],[4,7],[7,10],[10,13],[13,16],[16,19],[19,22],[22,25],[25,28]]
    if len(points) == 62:
        points = np.array(points).T
        for num in range(len(limb_ranges)):
            visible_limb = []
            for x in range(limb_ranges[num][0],limb_ranges[num][1]):
                if vis[x]== 1:
                    visible_limb.append(x) 
            if prediction:
                line, = ax.plot(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb],'--', alpha=0.7, label='Prediction', color='red')
                ax.scatter(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb], marker='X',s=10, color='red')
            else:
                line, = ax.plot(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb], label='Actual', color='blue')
                ax.scatter(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb], marker='o',s=4, color='blue')
        return ax, line
    elif len(points) == 28:
        points = np.array(points).T
        for num in range(len(reduced_limb_ranges)):
            visible_limb = []
            for x in range(reduced_limb_ranges[num][0],reduced_limb_ranges[num][1]):
                if vis[x]== 1:
                    visible_limb.append(x) 
            if prediction:
                line, = ax.plot(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb],'--', alpha=0.7, label='Prediction', color='red')
                ax.scatter(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb], marker='X',s=10, color='red')
            else:
                line, = ax.plot(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb], label='Actual', color='blue')
                ax.scatter(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb], marker='o',s=4, color='blue')
        return ax, line

def set_axes_equal(ax):
    # workaround, as matplotlib's 3D plot has no option for equisised axes (10/2021)
    x_limits = ax.get_xlim3d()
    y_limits = ax.get_ylim3d()
    z_limits = ax.get_zlim3d()

    x_range = abs(x_limits[1] - x_limits[0])
    x_middle = np.mean(x_limits)
    y_range = abs(y_limits[1] - y_limits[0])
    y_middle = np.mean(y_limits)
    z_range = abs(z_limits[1] - z_limits[0])
    z_middle = np.mean(z_limits)

    plot_radius = 0.5*max([x_range, y_range, z_range])

    ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius])
    ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
    ax.set_zlim3d([z_middle - plot_radius, z_middle + plot_radius])


In [3]:
file_path = '../data/single_sungaya/label_names.txt'

with open(file_path) as file:
    label_names = file.readlines()
    label_names = [item.rstrip() for item in label_names]


In [4]:
target_dir = "../data/single_sungaya/"
out_df = pd.read_hdf(os.path.join(target_dir, "Data_3D_Pose.hdf5"))
reduceKeypoints = True
sungaya_dataset_synth = src.bug_dataset.BugDataset(df=out_df,
                             root_dir=target_dir, reduced=reduceKeypoints, transform=transforms.Compose([
                                src.bug_dataset.ToTensor()
                                   ]))

In [5]:
# Sungaya Real
target_dir = "../data/real_sungaya/"
out_df = pd.read_csv(os.path.join(target_dir, "realdata.csv"))
sungaya_dataset = RealBugDataset(df=out_df, root_dir=target_dir, std=sungaya_dataset_synth.std_2d   , mean=sungaya_dataset_synth.means_2d  , transform=transforms.Compose([
                                src.real_bug_dataset.ToTensor()
                                   ]))


In [6]:

batch_size = 128
test_dataloader = DataLoader(sungaya_dataset, batch_size=batch_size)

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))   

if reduceKeypoints:
    model = LinearModel(2*28,28,1024) # Linear size 512 looked promising 256 too small
else:
    model = LinearModel(2*62,62,512)

model = model.to(device)
path = "../models/lifting_depth/lifting_depth_model_20220323-150326_128"
model.load_state_dict(torch.load(path))
model.eval()
trained_model = model

Using cuda device


In [8]:
with torch.no_grad():
    for data in test_dataloader:
        file_name = data['file_name']
        image = data['image']
        X = data['key_points_2D']
        vis = data['visibility']
        # print(y)
        X = X.to(device, dtype=torch.float)
        
        pred = trained_model(torch.flatten(X, start_dim=1, end_dim=2))
            

In [9]:
sample = 8

kp2d = ((X[sample].cpu().numpy())*sungaya_dataset.std_2d)+sungaya_dataset.means_2d
prediction = pred[sample].cpu().numpy()
visibility_mask = vis[sample]

sungaya_dataset
mean3d = sungaya_dataset_synth.means_3d[:,2]
std3d = sungaya_dataset_synth.std_3d[:,2]

fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(projection='3d')

# ax, line1 = plot_stick_bug(ax, unnormalised_acc, visibility_mask)


unnormalised_est = (prediction*std3d)+mean3d
est = np.hstack((kp2d,np.zeros((28,1))))
est[:,2] += unnormalised_est

ax, line2 = plot_stick_bug(ax, est, visibility_mask, True)

ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Z axis')
plt.title(file_name[sample])
plt.legend(handles=[line2], loc='upper right')

# use custom function to ensure equal axis proportions
set_axes_equal(ax)
# opens external plot
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
name = file_name[sample]

img = image[sample].cpu().numpy()
img=np.swapaxes(img,0,1)
img=np.swapaxes(img,1,2)
plt.figure(figsize=(8,8))
plt.imshow(img)
plt.title(name)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …